In [157]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
%matplotlib inline
print('Done!')

Done!


In [158]:
from sklearn.neighbors import KNeighborsRegressor
import scipy.stats
from sklearn.preprocessing import StandardScaler
from pycaret.regression import setup, compare_models
from sklearn.model_selection import KFold, cross_val_score

from catboost import CatBoostRegressor
from sklearn.linear_model import BayesianRidge, HuberRegressor, Ridge, OrthogonalMatchingPursuit
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

import optuna

In [159]:
Boston_Train = pd.read_csv('C:\\Users\\Auditor\\Desktop\\Victor\\New folder\\Data Science\\Projects\\Boston Housing\\train.csv')

In [160]:
Boston_Test = pd.read_csv('C:\\Users\\Auditor\\Desktop\\Victor\\New folder\\Data Science\\Projects\\Boston Housing\\test.csv')

In [161]:
Boston_Train

,ID,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
3,5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
4,7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311,15.2,395.60,12.43,22.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,500,0.17783,0.0,9.69,0,0.585,5.569,73.5,2.3999,6,391,19.2,395.77,15.10,17.5
329,502,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,9.67,22.4
330,503,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
331,504,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9


In [162]:
Boston_Train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ID       333 non-null    int64  
 1   crim     333 non-null    float64
 2   zn       333 non-null    float64
 3   indus    333 non-null    float64
 4   chas     333 non-null    int64  
 5   nox      333 non-null    float64
 6   rm       333 non-null    float64
 7   age      333 non-null    float64
 8   dis      333 non-null    float64
 9   rad      333 non-null    int64  
 10  tax      333 non-null    int64  
 11  ptratio  333 non-null    float64
 12  black    333 non-null    float64
 13  lstat    333 non-null    float64
 14  medv     333 non-null    float64
dtypes: float64(11), int64(4)
memory usage: 39.2 KB


In [163]:
Boston_Test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ID       173 non-null    int64  
 1   crim     173 non-null    float64
 2   zn       173 non-null    float64
 3   indus    173 non-null    float64
 4   chas     173 non-null    int64  
 5   nox      173 non-null    float64
 6   rm       173 non-null    float64
 7   age      173 non-null    float64
 8   dis      173 non-null    float64
 9   rad      173 non-null    int64  
 10  tax      173 non-null    int64  
 11  ptratio  173 non-null    float64
 12  black    173 non-null    float64
 13  lstat    173 non-null    float64
dtypes: float64(10), int64(4)
memory usage: 19.1 KB


In [164]:
target = Boston_Train['medv']
test_ids = Boston_Test['ID']

train1 = Boston_Train.drop(['ID', 'medv'], axis=1)
test1 = Boston_Test.drop('ID', axis=1)

data1 = pd.concat([train1, test1], axis=0).reset_index(drop=True)
data1

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14
2,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94
3,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33
4,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311,15.2,395.60,12.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.17899,0.0,9.69,0,0.585,5.670,28.8,2.7986,6,391,19.2,393.29,17.60
502,0.28960,0.0,9.69,0,0.585,5.390,72.9,2.7986,6,391,19.2,396.90,21.14
503,0.23912,0.0,9.69,0,0.585,6.019,65.3,2.4091,6,391,19.2,396.90,12.92
504,0.22438,0.0,9.69,0,0.585,6.027,79.7,2.4982,6,391,19.2,396.90,14.33


In [165]:
target

0      24.0
1      21.6
2      33.4
3      36.2
4      22.9
       ... 
328    17.5
329    22.4
330    20.6
331    23.9
332    11.9
Name: medv, Length: 333, dtype: float64

In [166]:
data2 = data1.copy()

In [167]:
data2

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14
2,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94
3,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33
4,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311,15.2,395.60,12.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.17899,0.0,9.69,0,0.585,5.670,28.8,2.7986,6,391,19.2,393.29,17.60
502,0.28960,0.0,9.69,0,0.585,5.390,72.9,2.7986,6,391,19.2,396.90,21.14
503,0.23912,0.0,9.69,0,0.585,6.019,65.3,2.4091,6,391,19.2,396.90,12.92
504,0.22438,0.0,9.69,0,0.585,6.027,79.7,2.4982,6,391,19.2,396.90,14.33


In [168]:
#Scaling¶

scaler = StandardScaler()
scaler.fit(data2)

data3 = pd.DataFrame(scaler.transform(data2), index=data2.index, columns=data2.columns)

In [169]:
data3

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat
0,-0.419782,0.284830,-1.287909,-0.272599,-0.144217,0.413672,-0.120013,0.140214,-0.982843,-0.666608,-1.459000,0.441052,-1.075562
1,-0.417339,-0.487722,-0.593381,-0.272599,-0.740262,0.194274,0.367166,0.557160,-0.867883,-0.987329,-0.303094,0.441052,-0.492439
2,-0.416750,-0.487722,-1.306878,-0.272599,-0.835284,1.016303,-0.809889,1.077737,-0.752922,-1.106115,0.113032,0.416163,-1.361517
3,-0.412482,-0.487722,-1.306878,-0.272599,-0.835284,1.228577,-0.511180,1.077737,-0.752922,-1.106115,0.113032,0.441052,-1.026501
4,-0.410243,0.048772,-0.476654,-0.272599,-0.265154,-0.388411,-0.070229,0.839244,-0.523001,-0.577519,-1.505237,0.426798,-0.031268
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,-0.399688,-0.487722,-0.211099,-0.272599,0.261784,-0.875644,-1.414418,-0.473678,-0.408041,-0.102376,0.344213,0.401471,0.693431
502,-0.386816,-0.487722,-0.211099,-0.272599,0.261784,-1.274549,0.153803,-0.473678,-0.408041,-0.102376,0.344213,0.441052,1.189646
503,-0.392690,-0.487722,-0.211099,-0.272599,0.261784,-0.378438,-0.116457,-0.658834,-0.408041,-0.102376,0.344213,0.441052,0.037418
504,-0.394406,-0.487722,-0.211099,-0.272599,0.261784,-0.367041,0.395615,-0.616479,-0.408041,-0.102376,0.344213,0.441052,0.235063


In [170]:
#Split Data¶

train_final = data3.loc[:Boston_Train.index.max(), :].copy()
test_final = data3.loc[Boston_Train.index.max() + 1:, :].reset_index(drop=True).copy()

In [171]:
train_final

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat
0,-0.419782,0.284830,-1.287909,-0.272599,-0.144217,0.413672,-0.120013,0.140214,-0.982843,-0.666608,-1.459000,0.441052,-1.075562
1,-0.417339,-0.487722,-0.593381,-0.272599,-0.740262,0.194274,0.367166,0.557160,-0.867883,-0.987329,-0.303094,0.441052,-0.492439
2,-0.416750,-0.487722,-1.306878,-0.272599,-0.835284,1.016303,-0.809889,1.077737,-0.752922,-1.106115,0.113032,0.416163,-1.361517
3,-0.412482,-0.487722,-1.306878,-0.272599,-0.835284,1.228577,-0.511180,1.077737,-0.752922,-1.106115,0.113032,0.441052,-1.026501
4,-0.410243,0.048772,-0.476654,-0.272599,-0.265154,-0.388411,-0.070229,0.839244,-0.523001,-0.577519,-1.505237,0.426798,-0.031268
...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,-0.399823,-0.487722,-0.211099,-0.272599,0.261784,-1.019535,0.175139,-0.663208,-0.408041,-0.102376,0.344213,0.428662,0.342996
329,-0.413229,-0.487722,0.115738,-0.272599,0.158124,0.439316,0.018673,-0.625796,-0.982843,-0.803212,1.176466,0.387217,-0.418147
330,-0.415249,-0.487722,0.115738,-0.272599,0.158124,-0.234548,0.288933,-0.716639,-0.982843,-0.803212,1.176466,0.441052,-0.500850
331,-0.413447,-0.487722,0.115738,-0.272599,0.158124,0.984960,0.797449,-0.773684,-0.982843,-0.803212,1.176466,0.441052,-0.983048


In [172]:
test_final

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat
0,-0.417342,-0.487722,-0.593381,-0.272599,-0.740262,1.282714,-0.265812,0.557160,-0.867883,-0.987329,-0.303094,0.396427,-1.208727
1,-0.417044,-0.487722,-1.306878,-0.272599,-0.835284,0.207096,-0.351157,1.077737,-0.752922,-1.106115,0.113032,0.410571,-1.043322
2,-0.403696,0.048772,-0.476654,-0.272599,-0.265154,-0.160466,0.978808,1.024638,-0.523001,-0.577519,-1.505237,0.441052,0.910700
3,-0.395935,0.048772,-0.476654,-0.272599,-0.265154,-0.931206,1.117494,1.087196,-0.523001,-0.577519,-1.505237,0.328448,2.421774
4,-0.400729,0.048772,-0.476654,-0.272599,-0.265154,-0.399808,0.616090,1.329635,-0.523001,-0.577519,-1.505237,0.329325,0.623344
...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,-0.399688,-0.487722,-0.211099,-0.272599,0.261784,-0.875644,-1.414418,-0.473678,-0.408041,-0.102376,0.344213,0.401471,0.693431
169,-0.386816,-0.487722,-0.211099,-0.272599,0.261784,-1.274549,0.153803,-0.473678,-0.408041,-0.102376,0.344213,0.441052,1.189646
170,-0.392690,-0.487722,-0.211099,-0.272599,0.261784,-0.378438,-0.116457,-0.658834,-0.408041,-0.102376,0.344213,0.441052,0.037418
171,-0.394406,-0.487722,-0.211099,-0.272599,0.261784,-0.367041,0.395615,-0.616479,-0.408041,-0.102376,0.344213,0.441052,0.235063


In [173]:
from sklearn.preprocessing import StandardScaler

X = train_final
y = Boston_Train['medv']

#scaler = StandardScaler()
#X_data = scaler.fit_transform(X)
#y_data = y.to_numpy()

In [174]:
X

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat
0,-0.419782,0.284830,-1.287909,-0.272599,-0.144217,0.413672,-0.120013,0.140214,-0.982843,-0.666608,-1.459000,0.441052,-1.075562
1,-0.417339,-0.487722,-0.593381,-0.272599,-0.740262,0.194274,0.367166,0.557160,-0.867883,-0.987329,-0.303094,0.441052,-0.492439
2,-0.416750,-0.487722,-1.306878,-0.272599,-0.835284,1.016303,-0.809889,1.077737,-0.752922,-1.106115,0.113032,0.416163,-1.361517
3,-0.412482,-0.487722,-1.306878,-0.272599,-0.835284,1.228577,-0.511180,1.077737,-0.752922,-1.106115,0.113032,0.441052,-1.026501
4,-0.410243,0.048772,-0.476654,-0.272599,-0.265154,-0.388411,-0.070229,0.839244,-0.523001,-0.577519,-1.505237,0.426798,-0.031268
...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,-0.399823,-0.487722,-0.211099,-0.272599,0.261784,-1.019535,0.175139,-0.663208,-0.408041,-0.102376,0.344213,0.428662,0.342996
329,-0.413229,-0.487722,0.115738,-0.272599,0.158124,0.439316,0.018673,-0.625796,-0.982843,-0.803212,1.176466,0.387217,-0.418147
330,-0.415249,-0.487722,0.115738,-0.272599,0.158124,-0.234548,0.288933,-0.716639,-0.982843,-0.803212,1.176466,0.441052,-0.500850
331,-0.413447,-0.487722,0.115738,-0.272599,0.158124,0.984960,0.797449,-0.773684,-0.982843,-0.803212,1.176466,0.441052,-0.983048


In [175]:
y

0      24.0
1      21.6
2      33.4
3      36.2
4      22.9
       ... 
328    17.5
329    22.4
330    20.6
331    23.9
332    11.9
Name: medv, Length: 333, dtype: float64

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import log_loss



# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a pipeline with a RandomForestClassifier
model = Pipeline([
    ('scaler', StandardScaler()),  # Optional: Standardize your features
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Fit the model on the training data
model.fit(X_train, y_train)

# Predict the target labels and probabilities for the test set
predicted_labels = model.predict(X_test)
predicted_probabilities = model.predict_proba(X_test)

# Calculate log loss
logloss = log_loss(y_test, predicted_probabilities)

# Display log loss
print(f"Log Loss: {logloss}")

# Display classification report
from sklearn.metrics import classification_report
print("Classification Report:")
print(classification_report(y_test, predicted_labels))

In [176]:
catboost_params = {
    'iterations': 6000,
    'learning_rate': 0.005,
    'depth': 4,
    'l2_leaf_reg': 1,
    'eval_metric':'RMSE',
    'early_stopping_rounds': 200,
    'random_seed': 42
}

br_params = {
    'n_iter': 304,
    'tol': 0.16864712769300896,
    'alpha_1': 5.589616542154059e-07,
    'alpha_2': 9.799343618469923,
    'lambda_1': 1.7735725582463822,
    'lambda_2': 3.616928181181732e-06
}

lightgbm_params = {
    'num_leaves': 39,
    'max_depth': 2,
    'learning_rate': 0.13705339989856127,
    'n_estimators': 273
}

ridge_params = {
    'alpha': 631.1412445239156
}

In [177]:
models = {
    "catboost": CatBoostRegressor(**catboost_params, verbose=0),
    "br": BayesianRidge(**br_params),
    "lightgbm": LGBMRegressor(**lightgbm_params),
    "ridge": Ridge(**ridge_params),
    "omp": OrthogonalMatchingPursuit()
}

In [178]:
for name, model in models.items():
    model.fit(train_final, target)
    print(name + " trained.")

catboost trained.
br trained.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000261 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 831
[LightGBM] [Info] Number of data points in the train set: 333, number of used features: 13
[LightGBM] [Info] Start training from score 22.768769
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wi

In [179]:
#Evaluating the results

In [180]:
results = {}

kf = KFold(n_splits=10)

for name, model in models.items():
    result = (-cross_val_score(model, train_final, target, scoring='neg_mean_squared_error', cv=kf))
    results[name] = result

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000327 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 749
[LightGBM] [Info] Number of data points in the train set: 299, number of used features: 13
[LightGBM] [Info] Start training from score 22.968562
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000310 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 755
[LightGBM] [Info] Number of data points in the train set: 300, number of used features: 13
[LightGBM] [Info] Start training from score 23.208333
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

In [181]:
results

{'catboost': array([  4.3539665 ,   5.8525785 ,   5.64433623,  22.24074502,
         12.29430326,   7.7887556 ,  11.66049578, 117.72197595,
         19.47408143,   9.69635829]),
 'br': array([  7.01135351,  14.83612529,  19.0887988 ,  38.12620958,
         33.13244948,  21.47341953,   9.07808337, 204.75549805,
         63.54293131,  12.25935375]),
 'lightgbm': array([  3.46248677,   4.76414417,   4.86011491,  24.06834744,
         18.06642699,  10.5303077 ,  14.89723713, 114.3353637 ,
         18.41841475,  10.6372881 ]),
 'ridge': array([ 22.34774129,  11.15593336,   9.04921574,  86.41829676,
         71.19258099,  58.68949117,  18.14568328, 143.29491223,
         48.52770822,   9.22935109]),
 'omp': array([ 15.87557495,  19.71528803,  24.41645462,  66.65397704,
         72.83993221,  66.36474094,  35.00617178, 234.16861436,
        122.9342031 ,  15.82745348])}

In [182]:
for name, result in results.items():
    print("----------\n" + name)
    print(np.mean(result))
    print(np.std(result))

----------
catboost
21.672759655220517
32.49944378144073
----------
br
42.33042226586002
56.500178497789705
----------
lightgbm
22.404013165556684
31.30873683514789
----------
ridge
47.80509141347315
41.395985434343736
----------
omp
67.38024105058183
64.35239442758585


In [183]:
#Combine Predictions¶

In [187]:
final_predictions = (
    0.4 * (models['catboost'].predict(test_final)) +
    0.2 * (models['br'].predict(test_final)) +
    0.2 * (models['lightgbm'].predict(test_final)) +
    0.1 * (models['ridge'].predict(test_final)) +
    0.1 * (models['omp'].predict(test_final))
)

In [188]:
final_predictions

array([32.70503996, 27.00874936, 16.751117  , 12.60395294, 18.39382361,
       18.7130923 , 20.10622479, 16.23921359, 15.38562868, 17.28605546,
       18.44842583, 20.12405687, 13.44405248, 15.04638904, 22.94804936,
       22.63349824, 23.90359953, 29.23759508, 11.63548813, 26.72018361,
       22.44747793, 23.96287943, 22.0076349 , 22.04268353, 21.56205944,
       22.83012751, 24.76048437, 23.57849793, 25.43516831, 26.30132481,
       37.35068276, 39.22946417, 32.60612248, 21.25797796, 18.06205633,
       21.58296361, 18.88540113, 17.7805674 , 19.35827224, 21.45047019,
       15.71894577, 20.33512514, 21.18591639, 17.50495474, 14.85655784,
       13.17066754, 12.1674068 , 17.9247256 , 20.42432548, 22.94305073,
       19.50282025, 33.29693036, 43.82367544, 22.56468218, 23.06181627,
       25.49510436, 30.50969666, 42.68479387, 36.29882682, 32.87080216,
       30.28964965, 39.87574497, 18.8122168 , 17.1265926 , 20.72100458,
       23.03653095, 24.98995587, 23.50521987, 20.64211112, 24.25

In [189]:
submission = pd.concat([test_ids, pd.Series(final_predictions, name='medv')], axis=1)
submission

,ID,medv
0,3,32.705040
1,6,27.008749
2,8,16.751117
3,9,12.603953
4,10,18.393824
...,...,...
168,496,18.741598
169,497,16.088468
170,499,20.858468
171,501,20.085902


In [190]:
submission.to_csv('C:\\Users\\Auditor\\Desktop\\Victor\\New folder\\Data Science\\Projects\\Boston Housing\\Final_submission.csv'
, index=False, header=True)